In [8]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [9]:
!pip install keras

In [0]:
!pip install -U -q PyDrive

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [14]:
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: train.csv, id: 1IVVyvPnt9NgqZvcIqx8XxM2S4kPS_h5b
title: Colab Notebooks, id: 1fQAst7DRe0d16b-aZ7pxd7TDbsqugtz7
title: genres.tar.gz, id: 12QuATACM0O7YYuseiRRl_t9eDWigZ3Gu
title: BIO DATA.docx, id: 0B4YR_BKQia_VTDVGaEZZbGpVRUpGaHNwZmlxVXo0RkN3U2Mw
title: product_train.csv, id: 1s27yNyxx04VcdwLftOLHwcKVpYWO4-Vt
title: product_test.csv, id: 15fkgFek2C09ic0SXYccfu3qIFkByNkBm
title: Bharath resume.doc, id: 1Inc1_YfX7reX-NlrncLl0eO4tYJTe_B3
title: 17M011_Bharath_GBM Assignment_India's forign trade.docx, id: 1pXLeZNJ_WabPsF-jiyx2E4xyC6AJE417
title: Untitled form, id: 1wuYGZtxDXD5zIErNxjZ3VJKY2yTS9uhaMllF_Xt4siI
title: PCG Contact Mail, id: 1BL3wTS-mYZ1H3gcH_0ymI-ByUTsjK2tHAbKr0Jf-JMw
title: PCG Contact Mail, id: 1fJwAQgXqyU4LANanHOxAu_B8QxKWLzwiCp1g2VUijr4
title: IMG_6219.JPG, id: 1FC6e8SMkCn5CJCFfJwecqEZz_aghyp0l
title: IMG_6221.JPG, id: 10uoT2K5YbJMpcv62TjP9dEIBbrJZN6_V
title: IMG_6207.JPG, id: 19PFtFW7dNwyiMK3feplc8RLlF-198eJi
title: IMG_6126.JPG, id: 1DWfd6RTqRFmQ7E4hDj0k2fNk_DB-Ti

In [0]:
downloaded = drive.CreateFile({'id': '1IVVyvPnt9NgqZvcIqx8XxM2S4kPS_h5b'})
downloaded.GetContentFile('train.csv')

In [16]:
import os
os.listdir()

['.config', 'adc.json', 'genres.tar.gz', 'drive', 'train.csv', 'sample_data']

In [0]:
import pandas as pd
all_data = pd.read_csv("train.csv")

In [18]:
all_data.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [0]:
import numpy as np
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(all_data['question_text'], all_data['target'], test_size=0.2, random_state=42)

In [21]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
from nltk import word_tokenize

In [23]:
question_lens=[]
for question in all_data['question_text']:
    question_lens.append(len(word_tokenize(question)))

max(question_lens)

np.quantile(question_lens,0.90)

from keras.preprocessing.text import Tokenizer

max_len = 30
tok = Tokenizer(char_level=False,split=' ')
tok.fit_on_texts(X_train)
sequences_train = tok.texts_to_sequences(X_train)

X_train.head()

vocab_len=len(tok.index_word.keys())

vocab_len

from keras.preprocessing import sequence

sequences_matrix_train = sequence.pad_sequences(sequences_train,maxlen=max_len)

sequences_matrix_train

sequences_matrix_train[0]

Using TensorFlow backend.


array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     9,     3,  4780, 11233,     3,   130,
          27,   427, 11233], dtype=int32)

In [0]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(vocab_len+1,300,input_length=max_len,mask_zero=True)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [0]:
from keras.layers import Input, Embedding, LSTM, Activation, Dense, Dropout
from keras.models import Model

In [26]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer='adam',
              metrics=['accuracy'])

W0826 15:31:15.241332 139829747267456 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0826 15:31:15.261754 139829747267456 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0826 15:31:15.263402 139829747267456 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0826 15:31:15.514152 139829747267456 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:2974: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instruct

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 30)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 30, 300)           58857900  
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                93440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257       
__________

In [0]:
sequences_test = tok.texts_to_sequences(X_test)
sequences_matrix_test = sequence.pad_sequences(sequences_test,
                                               maxlen=max_len)

In [28]:
model.fit(sequences_matrix_train,y_train,batch_size=500,epochs=10,validation_data=(sequences_matrix_test,y_test))

Train on 1044897 samples, validate on 261225 samples
Epoch 1/10
1044897/1044897 [==============================] - 341s 326us/step - loss: 0.1252 - acc: 0.9520 - val_loss: 0.1112 - val_acc: 0.9556
Epoch 2/10
1044897/1044897 [==============================] - 336s 322us/step - loss: 0.0972 - acc: 0.9604 - val_loss: 0.1108 - val_acc: 0.9565
Epoch 3/10
1044897/1044897 [==============================] - 336s 322us/step - loss: 0.0799 - acc: 0.9673 - val_loss: 0.1218 - val_acc: 0.9542
Epoch 4/10
1044897/1044897 [==============================] - 335s 321us/step - loss: 0.0638 - acc: 0.9739 - val_loss: 0.1438 - val_acc: 0.9510
Epoch 5/10
1044897/1044897 [==============================] - 335s 321us/step - loss: 0.0502 - acc: 0.9793 - val_loss: 0.1711 - val_acc: 0.9524
Epoch 6/10
1044897/1044897 [==============================] - 336s 321us/step - loss: 0.0394 - acc: 0.9837 - val_loss: 0.1998 - val_acc: 0.9496
Epoch 7/10
1044897/1044897 [==============================] - 335s 321us/step - los

In [0]:
predictions=model.predict(sequences_matrix_test)

In [0]:
from sklearn.metrics import roc_auc_score

In [31]:
roc_auc_score(y_test,predictions)

0.9164755781710905